# Place for experimenting the progressive design

In [383]:
import os,sys
import yaml
import inspect
import importlib

sys.path.append('..')

import model_discovery.utils as U
from model_discovery.configs.gam_config import GAMConfig, GAMConfig_14M
from model_discovery.model.composer import GAUTree,GAUBase,GAUNode
# from model_discovery.evolution import BuildEvolution
import model_discovery.agents.flow.gau_utils
importlib.reload(model_discovery.agents.flow.gau_utils)
from model_discovery.agents.flow.gau_utils import check_and_reformat_gau_code

ckpt_dir = os.environ['CKPT_DIR']
lib_dir = U.pjoin(ckpt_dir, 'test_composer', 'lib')
test_tree = GAUTree('TestTree', None, None, None,None,lib_dir)

prompts_dir='../model_discovery/agents/prompts/'
gab_py = U.read_file(U.pjoin(prompts_dir,'gab_template.py'))
gam_py = U.read_file(U.pjoin(prompts_dir,'gam_prompt.py'))
GAU_TEMPLATE = U.read_file(U.pjoin(prompts_dir,'gau_template.py'))
GAU_BASE=inspect.getsource(GAUBase)


## Parsers

In [384]:
import json

code='''
# gau.py   # DO NOT CHANGE OR REMOVE THE MAKK HERE, KEEP IT ALWAYS THE FIRST LINE #

import torch
import torch.nn as nn

from model_discovery.model.utils.modules import GAUBase # DO NOT CHANGE THIS IMPORT STATEMENT #


# YOU CAN IMPORT MORE MODULES HERE #

# YOU CAN DEFINE MORE CLASSES OR FUNCTIONS HERE #

# Placeholder classes for future implementation
class MemoryAccessUnit(nn.Module):
    def __init__(self, embed_dim, memory_size, device=None, dtype=None):
        super().__init__(embed_dim)

    def _forward(self, X, **Z):
        return X, {}

class DownsamplingUnit(nn.Module):
    def __init__(self, embed_dim, downsample_factor, device=None, dtype=None):
        super().__init__(embed_dim)

    def _forward(self, X, **Z):
        return X, {}

class GAU(GAUBase): # DO NOT CHANGE THE NAME OF THIS CLASS
    """Generalized Autoregressive Block Unit
        Input:        X: (batch, seqlen, embed_dim), Z: {dict of all current intermediate variables}
        Output:       Y: (batch, seqlen, embed_dim), Z_: Optional, {dict of *new* intermediate variables to update the current Z}
        Constraints:  Causal, differentiable, parameter number, complexity, parallelizable
    """
    def __init__(self, embed_dim: int, device=None, dtype=None,**kwargs): # YOU CAN ADD MORE ARGUMENTS WITH OPTIONAL DEFAULT VALUES, BUT YOU HAVE TO HAVE embed_dim, device, dtype AS THE ARGUTMENTS #
        # argv: list of hyperparameters
        factory_kwargs = {"device": device, "dtype": dtype} # remember to pass it to all nn layers
        super().__init__(embed_dim) # DO NOT CHANGE THIS LINE #

        # COMPLETING THE CODE HERE #
        self.token_scorer = TokenScoringGAU(embed_dim, **factory_kwargs)
        self.dual_path: GAUBase = DualPathGAU(embed_dim, **factory_kwargs)
        self.latent_attention: GAUBase = LatentAttentionGAU(embed_dim, **factory_kwargs)

        self.stages: nn.ModuleList = nn.ModuleList([CompressionStageGAU(embed_dim, device=
            device, dtype=dtype, **kwargs) for _ in range(2)])

        self.model = nn.Sequential(
          nn.Conv2d(1,20,5),
          nn.ReLU(),
          LatentAttentionGAU(),
          nn.Conv2d(20,64,5),
          nn.ReLU(),
          nn.Sequential(OrderedDict([
                ('conv1', nn.Conv2d(1, 20, 5)),
                ('relu1', nn.ReLU()),
                ('gau2', DualPathGAU()),
                ('conv2', nn.Conv2d(20, 64, 5)),
                ('relu2', nn.ReLU())
            ]))
        )

        self.activations = nn.ModuleDict({
            'relu': nn.ReLU(),
            'lrelu': nn.LeakyReLU(),
            'gau3': CompressionStageGAU()
        })

        self.model2 = nn.Sequential(OrderedDict([
          ('conv1', nn.Conv2d(1,20,5)),
          ('relu1', nn.ReLU()),
          ('gau2', DualPathGAU())
          ('conv2', nn.Conv2d(20,64,5)),
          ('relu2', nn.ReLU())
        ]))

    # YOU CAN ADD MORE FUNCTIONS HERE #


    def _forward(self, X, **Z): 

        # THE CODE HERE MUST BE COMPLETED #
        # Step 1: Score tokens
        X, Z = self.token_scorer(X, **Z)
        # Step 2: Route through dual paths
        # Step 3: Apply latent attention
        Y, Z = self.latent_attention(X, **Z)
        X, Z = self.dual_path(X, **Z)

        return Y, Z
'''
children = ['TokenScoringGAU', 'DualPathGAU', 'LatentAttentionGAU','CompressionStageGAU'] # solve redundancy 

In [385]:

unit_name = "XAU"  # Provide the unit_name to rename GAU class
reformatted_code, new_args, errors, warnings = check_and_reformat_gau_code(code, unit_name,children)
print("Reformatted Code:\n" + reformatted_code)
print("Errors:\n", errors)
print("Warnings:\n", warnings)
print("New Arguments:\n", new_args)

# test_tree.add_unit(
#     unit_name,reformatted_code,new_args,None,called,None,None,children_units,None
# )
# test_tree.root=test_tree.units['XAU']

Rewritten instantiation: TokenScoringGAU(embed_dim, block_loc, kwarg_all, **self.factory_kwargs, **kwarg_all)
Rewritten instantiation: DualPathGAU(embed_dim, block_loc, kwarg_all, **self.factory_kwargs, **kwarg_all)
Rewritten instantiation: LatentAttentionGAU(embed_dim, block_loc, kwarg_all, **self.factory_kwargs, **kwarg_all)
Rewritten instantiation: CompressionStageGAU(embed_dim, block_loc, kwarg_all, **self.factory_kwargs, **kwarg_all)
Rewritten instantiation: CompressionStageGAU(embed_dim, block_loc, kwarg_all, **self.factory_kwargs, **kwarg_all)
Reformatted Code:
import torch
import torch.nn as nn
from model_discovery.model.utils.modules import GAUBase


class MemoryAccessUnit(nn.Module):

    def __init__(self, embed_dim, memory_size, device=None, dtype=None):
        super().__init__(embed_dim)

    def _forward(self, X, **Z):
        return X, {}


class DownsamplingUnit(nn.Module):

    def __init__(self, embed_dim, downsample_factor, device=None, dtype=None):
        super().

In [386]:
children={
    'unit1': 'MockUnit1',
    'unit2': 'MockUnit2',
}
called=['unit1', 'unit2', 'unit1']
test_tree.units['DualPathGAU'] = GAUNode('DualPathGAU', None, None, None, called, None, None, children, None)
children={
    'unit1': 'MockUnit3',
    'unit2': 'MockUnit4',
}
called=['unit2', 'unit2']
test_tree.units['MockUnit1'] = GAUNode('MockUnit1', None, None, None, called, None, None, children, None)

In [387]:
test_tree.units

{'DualPathGAU': GAUNode(name='DualPathGAU', code=None, args=None, desc=None, path=['unit1', 'unit2', 'unit1'], review=None, rating=None, children={'unit1': 'MockUnit1', 'unit2': 'MockUnit2'}, suggestions=None),
 'MockUnit1': GAUNode(name='MockUnit1', code=None, args=None, desc=None, path=['unit2', 'unit2'], review=None, rating=None, children={'unit1': 'MockUnit3', 'unit2': 'MockUnit4'}, suggestions=None)}

In [ ]:
class A:
    def __init__(self,r,d,e,**Z):
        print('Args in A:',d,e)
        print('Kwargs in A:',Z)

class B(A):
    def __init__(self,_Z,c=1,d=2,e=3,**Z):
        args2={'d':d,'e':e}
        super().__init__(c,**_Z,**args2)
        print('Kwargs in B:',Z)

cfg={
    'a':1,
    'b':2,
    'c':3,
}

B(cfg,**cfg)

Args in A: 2 3
Kwargs in A: {'a': 1, 'b': 2, 'c': 3}
Kwargs in B: {'a': 1, 'b': 2}


In [395]:
def A(X,Z,**kwargs):
    print(X)
    print(Z)
    print(kwargs)

def B(X,Z,a,**kwargs):
    A(X,Z,**Z)

Z={'a':3}
B(1,Z,**Z)

1
{'a': 3}
{'a': 3}
